In [ ]:
import initialize_notebook # noqa

# Tool Calling

In [ ]:
import json
import random

from hslu.dlm03.common import backend

In [ ]:
model_client, model_name = backend.Gemini2p5Flash().get_client()

# Tool Definitions

In [ ]:
def multiply(a: float, b: float) -> float:
    return a * b


def add(a: float, b: float) -> float:
    return a + b

In [ ]:
multiply_tool = dict(
    type="function",
    function=dict(
        name="multiply",
        description="Multiplies 2 numbers together",
        parameters={
            "type": "object",
            "properties": {
                "a": {
                    "type": "number",
                },
                "b": {
                    "type": "number",
                },
            },
            "required": [
                "a",
                "b",
            ],
        },
        strict=True,
    ),
)
add_tool = dict(
    type="function",
    function=dict(
        name="add",
        description="Adds 2 numbers together",
        parameters={
            "type": "object",
            "properties": {
                "a": {
                    "type": "number",
                },
                "b": {
                    "type": "number",
                },
            },
            "required": [
                "a",
                "b",
            ],
        },
        strict=True,
    ),
)


# Agentic Environment

In [ ]:
def agentic_environment(tool_call):
    tool_name = tool_call.function.name
    tool_arguments = json.loads(tool_call.function.arguments)
    call_id = tool_call.id
    match tool_name:
        case "multiply":
            return {
                "role": "tool",
                "tool_call_id": call_id,
                "content": str(multiply(**tool_arguments)),
            }
        case "add":
            return {
                "role": "tool",
                "tool_call_id": call_id,
                "content": str(add(**tool_arguments)),
            }
        case _:
            return {
                "role": "tool",
                "tool_call_id": call_id,
                "content": f"Unknown tool {tool_name}",
            }


# Agentic Loop

In [ ]:
QUERY = "What is (2 + 3) * 4?"

messages = [
    {"role": "user", "content": QUERY},
]
done = False
while not done:
    response = model_client.chat.completions.create(
        messages=messages,
        model=model_name,
        tools=[multiply_tool, add_tool],
    )
    done = True
    message = random.choice(response.choices).message
    messages.append(message)
    print(message)
    if message.content:
        print(message.content)
    if message.tool_calls:
        for tool_call in message.tool_calls:
            done = False
            tool_call_result = agentic_environment(tool_call)
            print(tool_call_result)
            messages.append(tool_call_result)